In [11]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import tensorflow as tf
tf.set_random_seed(2)

config = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1 
)

config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
sess = tf.Session(graph= tf.get_default_graph(), config=config) 
    
import keras 
from keras import backend as K
from keras.models import load_model

In [7]:
projectspace = '/vol/vssp/AcousticEventsDetection/DLGdansk/UrbanSound/'
modelpath = os.path.join(projectspace,'models','crnn_epoch_156_val_loss_1.3890.hdf5')

In [12]:
model=load_model(modelpath)

In [17]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
layer_dict.keys()

[u'conv2d_4',
 u'reshape_1',
 u'max_pooling2d_4',
 u'max_pooling2d_1',
 u'dense_1',
 u'max_pooling2d_3',
 u'max_pooling2d_2',
 u'lstm_1',
 u'batch_normalization_4',
 u'input_1',
 u'batch_normalization_2',
 u'batch_normalization_3',
 u'batch_normalization_1',
 u'conv2d_1',
 u'conv2d_3',
 u'conv2d_2']

In [18]:
layer_name = 'conv2d_1'
filter_index = 0

In [20]:
# build a loss function that maximizes the activation
# of the nth filter of the layer considered
layer_output = layer_dict[layer_name].output
loss = K.mean(layer_output[:, :, :, filter_index])

# compute the gradient of the input picture wrt this loss
grads = K.gradients(loss, input_img)[0]

# normalization trick: we normalize the gradient
grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

# this function returns the loss and grads given the input picture
iterate = K.function([input_img], [loss, grads])

NameError: name 'input_img' is not defined